 # Food Classification Using Deep Neural Networks and Transfer Learning

# 1. Data Splitting 

Split the data into training, validation, and test sets. Justify your choice.

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

import matplotlib.pyplot as plt


# Load Data

In [ ]:
def get_data_loader(target_classes, batch_size):
    """ Returns the indices for datapoints in the dataset that
    belongs to the desired target classes, a subset of all possible classes.

    Args:
        dataset: Dataset object
        classes: A list of strings denoting the name of each class
        target_classes: A list of strings denoting the name of the desired
                        classes. Should be a subset of the argument 'classes'
    Returns:
        indices: list of indices that have labels corresponding to one of the
                 target classes
    """
    classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels')


    transform = transforms.Compose(
    [transforms.Resize((224,224), interpolation=2), transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.ImageFolder(root = 'train',
                                                 transform=transform)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=transform)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'test',
                                            transform=transform)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)

    return train_loader, classes#, val_loader, test_loader, classes

In [ ]:
classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels')

transform = transforms.Compose(
    [transforms.Resize((224,224), interpolation=2), transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 64,
                                          num_workers = 1, shuffle = True)
val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64,
                                              num_workers=1, shuffle = True)

testset = torchvision.datasets.ImageFolder(root= 'test',
                                            transform=transform)
    
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                             num_workers=1, shuffle = True)

In [ ]:
trainset.class_to_idx


In [ ]:
train_loader_iter = iter(train_loader)
imgs, labels = next(train_loader_iter)
print(imgs.shape)
print(labels)
print(labels.shape)

# 2. Convolutional Network - Baseline Model 
Build a convolutional neural network model that takes the (224x224 RGB) image as input.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

# 3. Training 
Train your network. Plot the training curve.

Make sure that you are checkpointing frequently!

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# convolutional neural network, 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 5, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 3, padding = 1)
        self.conv3 = nn.Conv2d(10, 15, 3, padding = 1)
        self.fc1 = nn.Linear(15 * 28 * 28 , 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 15 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path


In [ ]:
train_loader_iter = iter(train_loader)
imgs, labels = next(train_loader_iter)
print(imgs.shape)
print(labels)
print(labels.shape)

# Training - Baseline Model

In [ ]:
def evaluate(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0

    for i, data in enumerate(loader, 0):
        inputs, labels = data
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        total_err += pred.ne(labels.view_as(pred)).sum().item()
        total_loss += loss.item()
        total_epoch += len(labels)

    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)

    return err, loss

In [ ]:
def train(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
    transform = transforms.Compose(
        [transforms.Resize((224,224)), transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = torchvision.datasets.ImageFolder(root='train',
                                                 transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          num_workers=1, shuffle = True)
    
    train_loader_iter = iter(train_loader)
    imgs, labels = next(train_loader_iter)
    
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            
            inputs, labels = data
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

In [ ]:
# Training Curve
import matplotlib.pyplot as plt
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """

    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    plt.plot(range(1,16), train_err, label="Train")
    plt.plot(range(1,16), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,16), train_loss, label="Train")
    plt.plot(range(1,16), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

# Hyperparameter Seach - Baseline Model

In [ ]:
ne3 = LargeNet()

In [ ]:
train(ne3, batch_size = 64, learning_rate=0.001, num_epochs = 15)

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 64, learning_rate=0.1, num_epochs = 15)

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 32, learning_rate=0.001, num_epochs = 15)

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 32, learning_rate=0.01, num_epochs = 15)

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 32, learning_rate=0.1, num_epochs = 15)

In [ ]:
path = 'model_large_bs64_lr0.001_epoch14'
plot_training_curve(path)


# Test Classification Error

In [ ]:
net = LargeNet()
model_path = get_model_name(net.name, batch_size=64, learning_rate=0.001, epoch=5)
state = torch.load(model_path)
net.load_state_dict(state)

evaluate(net, test_loader, nn.CrossEntropyLoss())


In [ ]:
The test classification error was about 0.37.

# Test Accuracy

In [ ]:
def get_accuracy(model):
    data = testset
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=32):
        output = model(imgs) # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
net = LargeNet()
model_path = get_model_name(net.name, batch_size=64, learning_rate=0.001, epoch=5)
state = torch.load(model_path)
net.load_state_dict(state)




In [ ]:
get_accuracy(net)

# 5. Transfer Learning [16 pt]
For many image classification tasks, it is generally not a good idea to train a very large Deep Neural Network model from scratch due to the enormous compute requirements and lack of sufficient amounts of training data. One of the better option is to try using an existing model that performs a similar task to the one you need to solve. This method of utilizing a pre-trained network for other similar tasks is broadly termed â€œTransfer Learningâ€. In this assignment, we will use Transfer Learning to extract features from the hand gesture images. Then, train a smaller network to use these features as input and classify the hand gestures.

As you have learned from the CNN lecture, Convolution layers extract various features from the images which get utilized by the fully connected layers for correct classification. AlexNet architecture played a pivotal role in establishing Deep Neural Nets as a go-to tool for Image classification problems and we will use an imagenet pre-trained AlexNet model to extract features in this assignment.

# AlexNet

In [ ]:
# Importing relevant Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import os
import torchvision
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from torch.nn import functional as F
import copy
from torch.autograd import Variable


###############################################################################
# Feature Extraction using AlexNet pretrained model

class AlexNetFeatures(nn.Module):
    '''
    Class that loads AlexNet Feature Model ('Convolution layers') with imagenet trained weights
    
    input : image tensors with dimension Lx3x224x224
    
    output : feature tensor with dimension Lx256x6x6
    
    *L - Batch size
    
    '''
    
    def load_weights(self):
        an_builtin = torchvision.models.alexnet(pretrained=True) # Loads the pretrained model weights
        
        features_weight_i = [0, 3, 6, 8, 10]
        for i in features_weight_i:
            self.features[i].weight = an_builtin.features[i].weight
            self.features[i].bias = an_builtin.features[i].bias

    def __init__(self):
        super(AlexNetFeatures, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        self.load_weights() # Copies the weights to AlexNetFeatures model layers

    def forward(self, x):
        x = self.features(x)
        return x




In [ ]:
myfeature_model = AlexNetFeatures()

# I don't know how to save alex net features using the cache thingy to make training run faster

In [ ]:
for i, data in enumerate(train_loader, 0):

    inputs, labels = data
 
    features = myfeature_model(inputs)
for i, data in enumerate(val_loader, 0):

    inputs, labels = data
 
    features = myfeature_model(inputs)
for i, data in enumerate(test_loader, 0):

    inputs, labels = data
 
    features = myfeature_model(inputs)
   

# Part (b) [5 pt]
Build a convolutional neural network model that takes as input these AlexNet features, and makes a prediction. Your model should be a subclass of nn.Module.

Explain your choice of neural network architecture: how many layers did you choose? What types of layers did you use: fully-connected or convolutional? What about other decisions like pooling layers, activation functions, number of channels / hidden units in each layer?

# Network For AlexNet

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.name = "transferNet"
        self.conv1 = nn.Conv2d(256, 1, 3) 
        self.fc1 = nn.Linear(16, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, 16)
        x = F.relu(self.fc1(x))
     
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [ ]:
classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels')

transform = transforms.Compose(
    [transforms.Resize((224,224), interpolation=2), transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 1,
                                          num_workers = 1, shuffle = True)
val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=1,
                                              num_workers=1, shuffle = True)

testset = torchvision.datasets.ImageFolder(root= 'test',
                                            transform=transform)
    
test_loader = torch.utils.data.DataLoader(testset, batch_size=1,
                                             num_workers=1, shuffle = True)

# Training With AlexNet

In [ ]:
def evaluatealex(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0

    for i, data in enumerate(loader, 0):
        inputs, labels = data
        
        features = myfeature_model(inputs)
        outputs = net(features)
        loss = criterion(outputs, labels)
        pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        total_err += pred.ne(labels.view_as(pred)).sum().item()
        total_loss += loss.item()
        total_epoch += len(labels)

    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)

    return err, loss


In [ ]:
def trainalex(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
    transform = transforms.Compose(
        [transforms.Resize((224,224)), transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = torchvision.datasets.ImageFolder(root='train',
                                                 transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          num_workers=1, shuffle = True)
    
    train_loader_iter = iter(train_loader)
    imgs, labels = next(train_loader_iter)
    
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            
            inputs, labels = data
                      
            optimizer.zero_grad()
            features = myfeature_model(inputs)
           
            outputs = net(features)
          
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

# Hyperparameter Search With AlexNet

In [ ]:
net = Net()
trainalex(net, batch_size=1, learning_rate=0.001, num_epochs=10)

# Test Accuracy-Alex Net
Report the test accuracy of your best model. How does the test accuracy compare to part 4(d)?

In [ ]:
def get_accuracy(model):
    data = testset
    total = 0
    correct =0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=32):
        features = myfeature_model(imgs)
        output = net(features) # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
get_accuracy(net)